In [1]:
!pip install keras-tuner --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import keras_tuner as kt


In [4]:
import kagglehub

path = kagglehub.dataset_download("gpiosenka/sports-classification")
print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/sports-classification


In [5]:
BASE_DIR = "/kaggle/input/sports-classification"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VALID_DIR = os.path.join(BASE_DIR, "valid")
TEST_DIR = os.path.join(BASE_DIR, "test")

print("Train classes:", len(os.listdir(TRAIN_DIR)))
print("Validation classes:", len(os.listdir(VALID_DIR)))
print("Test classes:", len(os.listdir(TEST_DIR)))


Train classes: 100
Validation classes: 100
Test classes: 100


In [6]:
img_height, img_width = 128, 128
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 13492 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
Classes: ['air hockey', 'ampute football', 'archery', 'arm wrestling', 'axe throwing', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling', 'bike polo', 'billiards', 'bmx', 'bobsled', 'bowling', 'boxing', 'bull riding', 'bungee jumping', 'canoe slamon', 'cheerleading', 'chuckwagon racing', 'cricket', 'croquet', 'curling', 'disc golf', 'fencing', 'field hockey', 'figure skating men', 'figure skating pairs', 'figure skating women', 'fly fishing', 'football', 'formula 1 racing', 'frisbee', 'gaga', 'giant slalom', 'golf', 'hammer throw', 'hang gliding', 'harness racing', 'high jump', 'hockey', 'horse jumping', 'horse racing', 'horseshoe pitching', 'hurdles', 'hydroplane racing', 'ice climbing', 'ice yachting', 'jai alai', 'javelin', 'jousting', 'judo', 'lacrosse', 'log rolling', 'luge', 'motorcycle racing', 'mushing', 'nascar racing', 'olym

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [8]:
def build_model(hp):
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),

        layers.Conv2D(
            filters=hp.Choice("conv_1_filter", values=[32, 64], default=32),
            kernel_size=hp.Choice("conv_1_kernel", values=[3,5]),
            activation="relu"
        ),
        layers.MaxPooling2D(),

        layers.Conv2D(
            filters=hp.Choice("conv_2_filter", values=[64, 128], default=64),
            kernel_size=hp.Choice("conv_2_kernel", values=[3,5]),
            activation="relu"
        ),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(
            units=hp.Choice("dense_units", values=[64, 128], default=64),
            activation="relu"
        ),
        layers.Dense(len(class_names), activation="softmax")
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [9]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    directory="tuner_dir",
    project_name="sports_classification"
)

tuner.search(train_ds, validation_data=val_ds, epochs=5)


Trial 5 Complete [00h 01m 01s]
val_accuracy: 0.009999999776482582

Best val_accuracy So Far: 0.2879999876022339
Total elapsed time: 00h 06m 35s


In [10]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:")
print("Conv1 filters:", best_hps.get("conv_1_filter"))
print("Conv1 kernel:", best_hps.get("conv_1_kernel"))
print("Conv2 filters:", best_hps.get("conv_2_filter"))
print("Conv2 kernel:", best_hps.get("conv_2_kernel"))
print("Dense units:", best_hps.get("dense_units"))
print("Learning rate:", best_hps.get("learning_rate"))

model = tuner.hypermodel.build(best_hps)


Best hyperparameters:
Conv1 filters: 32
Conv1 kernel: 3
Conv2 filters: 128
Conv2 kernel: 5
Dense units: 128
Learning rate: 0.0001


In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.0307 - loss: 4.5181 - val_accuracy: 0.0680 - val_loss: 4.1890
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.1149 - loss: 3.9566 - val_accuracy: 0.1660 - val_loss: 3.5447
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.2290 - loss: 3.2730 - val_accuracy: 0.2160 - val_loss: 3.1740
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.3460 - loss: 2.7006 - val_accuracy: 0.2600 - val_loss: 2.9905
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.4263 - loss: 2.3035 - val_accuracy: 0.2620 - val_loss: 2.9858
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.5212 - loss: 1.9188 - val_accuracy: 0.2700 - val_loss: 3.0181
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.5950 - loss: 1.6135 - val_accuracy: 0.2980 - val_loss: 3.0125
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.6780 - loss: 1.3097 - 

In [12]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.3310 - loss: 3.0935
Test Accuracy: 0.30799999833106995


In [13]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
                       precision    recall  f1-score   support

           air hockey       0.50      0.40      0.44         5
      ampute football       0.33      0.20      0.25         5
              archery       0.25      0.20      0.22         5
        arm wrestling       0.43      0.60      0.50         5
         axe throwing       0.33      0.20      0.25         5
         ba

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
